In [1]:
import pandas as pd 
import numpy as np
import os 
import time
import glob

# Filter piRNA loci / filter antisense mapping reads

In [10]:
# first make bed with uni piRNA loci

In [20]:
%%bash

# one way to do this is with bedtools merge

module load python
source activate rnaseq_basic

reference=/fs/ess/PCON0160/ben/genomes/c_elegans/WS279/c_elegans.PRJNA13758.WS279.slim.bed

cat $reference |\
    #awk '($5=="gene")' |\
    #grep piRNA |\
    grep exon |\
    #egrep "piRNA|miRNA|protein_coding|pseudogene" |\
    awk -F'\t' '{OFS="\t"; print $1,$2-10,$3+10,$7,$8,$6}' |\
    awk -F'\t' '{OFS="\t"; if ($2<0) print $1,0,$3,$4,$5,$6; else print $0}' |\
    sort -k1,1 -k2,2n |\
    bedtools merge -c 1,4,5,6 -o count,collapse,collapse,collapse |\
    awk '($4==1)' |\
    grep piRNA |\
    awk -F'\t' '{OFS="\t"; if ($7=="+") print $1,$2+10,$3,$5,$6,$7; else print $1,$2,$3-10,$5,$6,$7}' \
    > c_elegans.PRJNA13758.WS279.distinct_piRNA_loci.bed

wc -l c_elegans.PRJNA13758.WS279.distinct_piRNA_loci.bed


The following have been reloaded with a version change:
  1) python/3.9-2022.05 => python/3.6-conda5.2



7206 c_elegans.PRJNA13758.WS279.distinct_piRNA_loci.bed


In [38]:
def uni_piRNA_loci(bed) : 
    
    """
    Make unique piRNA locus bedfile
    """
    
    chroms, starts, ends, genes, strands, biotypes = [], [], [], [], [], []
    with open(bed, 'r') as f : 
        for line in f : 
            info = line.strip().split("\t")
            chrom = str(info[0])
            feature = str(info[4])
            start = int(info[1])
            end = int(info[2])
            strand = str(info[5])
            gene = str(info[6])
            biotype = str(info[7])
            
            #if (feature == "gene" and biotype == "piRNA") or (feature == "gene" and biotype == "redefine_type2_piRNA") or (feature == "gene" and biotype == "miRNA") or (feature == "exon" and biotype == "protein_coding") or (feature == "exon" and biotype == "pseudogene") or (biotype == "lincRNA") or (biotype == "ncRNA") :
            if (feature == "gene" and biotype == "piRNA") or (feature == "gene" and biotype == "miRNA") or (feature == "exon" and biotype == "protein_coding") or (feature == "exon" and biotype == "pseudogene") or (biotype == "lincRNA") or (biotype == "ncRNA") :
                chroms.append(chrom)
                starts.append(start)
                ends.append(end)
                genes.append(gene)
                strands.append(strand)
                biotypes.append(biotype)
    f.close() 
    
    df = pd.DataFrame({
        'chrom':chroms,
        'start':starts,
        'end':ends,
        'gene':genes,
        'strand':strands,
        'biotype':biotypes
    })
    
    df['start'] = df['start']
    df['end'] = df['end']
    sort = df.sort_values(by = ['chrom', 'start'], ascending = True).reset_index(drop = True)
    
    sort.to_csv("sorted_bed.tsv", sep = "\t", header = True, index = False)
    idx = []
    
    #
    #   U------------>
    #      U-------------> 
    #          
    k = 0
    for i,row in sort.iterrows() :
        if i < len(sort)-1 :
            
            # check nearest right neighbor
            if int(sort.iloc[i+1]['start'])-10 > int(sort.iloc[i]['end'])+10 :
                
                # check nearest left neighbor
                if int(sort.iloc[i]['start'])-10 > int(sort.iloc[i-1]['end'])+10 :
                    
                    idx.append(i)
    
    uni = sort.iloc[idx]
    uni = uni.reset_index(drop = True)
    uni_piRNA = uni.query('biotype == "piRNA"').reset_index(drop = True).drop(columns = ['biotype'])
    return uni_piRNA
        

In [50]:
uni = uni_piRNA_loci("/fs/ess/PCON0160/ben/genomes/c_elegans/WS279/c_elegans.PRJNA13758.WS279.slim.bed")

In [51]:
uni.to_csv("uniq_piRNA.tsv", sep = "\t", header = False, index = False)

In [52]:
uni.shape[0]

7318

In [53]:
def read_bed(bedfile) : 
    
    d = pd.read_csv(bedfile, usecols = [0, 1, 2, 3, 7, 9, 10, 13], header = 0, names = ['chrom', 'start', 'end', 'seq', 'biotype', 'feature', 'gene', 'count'], sep = "\t")
    #d = pd.read_csv(bedfile, usecols = [0,1,2,3,9,10,14], header = 0, names = ['chrom', 'start', 'end', 'seq', 'feature', 'gene', 'count'], sep = "\t")

    sub = d.query('feature == "antipiRNA"').reset_index(drop = True)

    return sub

In [54]:
def filter_piRNA(df) : 
        
    antipiRNA = df.query(
        'feature == "antipiRNA" and (seq.str[0] == "G" or seq.str[0]=="A") and (seq.str.len()>=17 and seq.str.len()<=19)'
    ).reset_index(
        drop = True
    )
    
    sense = antipiRNA.query('strand == "+"').reset_index(drop = True)
    anti = antipiRNA.query('strand == "-"').reset_index(drop = True)
    
    # for antisense piRNA further filter by removing situations like this
    #           rs U-----------------> re
    #          s <----------G/A e

    #          rs <-----------------U re
    #                   s G/A------------> e
    
    filt_sense = sense.query('ref_start < start').reset_index(drop = True)
    
    filt_anti = anti.query('end < ref_end').reset_index(drop = True)
    
    anti_piRNA_filt = pd.concat([filt_sense, filt_anti], ignore_index = True)
    
    return anti_piRNA_filt

In [55]:
def process_piRNA(uniq_piRNA, BEDFILES) :
    
    uni = pd.read_csv(uniq_piRNA, names = ['chrom', 'start', 'end', 'gene','strand'], usecols = [0,1,2,3,4], sep = "\t")
    uni_piRNA = uni['gene'].tolist()
    
    uni = uni.rename(columns = {'start':'ref_start', 'end':'ref_end'})

    chroms = []
    starts = []
    ends = []
    seqs = []
    counts = []
    strands = []
    genes = []
    features = []
    for e,B in enumerate(BEDFILES) : 
        print(os.path.basename(B))
        d = read_bed(B)
        d['sample'] = os.path.basename(B).split(".")[0]

        d = d.merge(uni, on = ['gene', 'chrom'], how = 'inner')
        
        if e == 0 : 
            table = d
        else : 
            table = pd.concat([table, d])
            
    filt = filter_piRNA(table)
                
    return filt
             

In [69]:
#DIRS = ['/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_07102023/bed', 
#        '/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_07_14_2023/bed',
#        '/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_oxi_07_18_2023/bed',
#        '/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_SNAP_ligation_07_24_2023/bed']
#DIRS = ['/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_normalize_miRNA_07_31_2023/bed']
#DIRS = ['/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_07_14_2023/bed']
#DIRS = ['/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_type2_piRNA_08_20_2023/bed']

DIRS = ['/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_07102023/bed']

BEDFILES = []

for D in DIRS : 
    bed = glob.glob(f"{D}/*parn_*YD*.bed.tsv")
    BEDFILES += bed
    
    

In [70]:
outname = "antipiRNA_09222023"

filt = process_piRNA(uniq_piRNA = "./uniq_piRNA.tsv",
                  BEDFILES = BEDFILES)

filt.to_csv(f"./{outname}.tsv", sep = "\t", header = True, index = False)
#else : 
#    filt = pd.read_csv(f"./{outname}.tsv", sep = "\t")

parn_YD2.trimmed.uniq.xartifacts.xc.aligned.v0.m1.bed.tsv
parn_YD1.trimmed.uniq.xartifacts.xc.aligned.v0.m1.bed.tsv


In [71]:
filt.query('feature == "antipiRNA"').sort_values(by = ['count'], ascending = False)

,chrom,start,end,seq,biotype,feature,gene,count,sample,ref_start,ref_end,strand
1711,IV,6050117,6050135,GTGGAATTTGAGGAAACA,piRNA,antipiRNA,"WBGene00049633,H16O14.17,21ur-2732",7.181639,parn_YD1,6050108,6050129,+
3680,IV,14697883,14697902,AGATAGTGGGATTAGAATA,piRNA,antipiRNA,"WBGene00169779,Y57G11C.797,21ur-8259",6.320672,parn_YD2,14697890,14697911,-
196,IV,6050117,6050135,GTGGAATTTGAGGAAACA,piRNA,antipiRNA,"WBGene00049633,H16O14.17,21ur-2732",5.980328,parn_YD2,6050108,6050129,+
5160,IV,14697883,14697902,AGATAGTGGGATTAGAATA,piRNA,antipiRNA,"WBGene00169779,Y57G11C.797,21ur-8259",4.899103,parn_YD1,14697890,14697911,-
5056,IV,14580095,14580114,ACGAGTTTGCTTCACTAGA,piRNA,antipiRNA,"WBGene00048342,Y57G11B.57,21ur-635",4.843431,parn_YD1,14580101,14580122,-
...,...,...,...,...,...,...,...,...,...,...,...,...
3133,IV,5970851,5970870,AGTATGGGGTTCATTAAAT,piRNA,antipiRNA,"WBGene00048889,B0350.20,21ur-3556",0.048621,parn_YD2,5970858,5970879,-
3135,IV,5981461,5981480,AATGGGGCTTGATGGCCGT,piRNA,antipiRNA,"WBGene00047205,B0350.11,21ur-1749",0.048621,parn_YD2,5981471,5981492,-
3136,IV,5981462,5981479,ATGGGGCTTGATGGCCG,piRNA,antipiRNA,"WBGene00047205,B0350.11,21ur-1749",0.048621,parn_YD2,5981471,5981492,-
3140,IV,6034565,6034584,ATATATATTTCATCTCGAA,piRNA,antipiRNA,"WBGene00166051,C46G7.42,21ur-6086",0.048621,parn_YD2,6034571,6034592,-


In [72]:
filt[['sample', 'gene']].drop_duplicates().groupby(['sample'])['gene'].count()

sample
parn_YD1    1455
parn_YD2    1511
Name: gene, dtype: int64

In [73]:
filt[['sample', 'gene']].query('sample == "parn_YD1" or sample == "parn_YD2"').drop(columns = ['sample']).drop_duplicates(['gene']).shape

(1934, 1)

# 1. Measure 5' to 5' and 5' to 3' distance relative to 5' U

In [25]:
def measure_5p3p_distance(df) : 
    
    #  5' rs U---------------------> re
    #         3' s <-------------G/A e 5' 
    
    #                 rs <-----------------U re 5'
    #    5'  s G/A------------------> e
    
    df = df.dropna().reset_index(drop = True)
    
    df['5p5p'] = df.apply(lambda row: row['end'] - row['ref_start'] if row['strand'] == "+" else row['ref_end'] - row['start'], axis = 1 )
    
    df['5p3p'] = df.apply(lambda row: row['start'] - row['ref_start'] if row['strand'] == "+" else row['ref_end'] - row['end'], axis = 1 )
    
    p5p5 = df.groupby(['5p5p', 'sample']).agg( { 'count':'sum' } ).reset_index()
    p5p5 = p5p5.rename(columns = {'5p5p' : 'distance'})
    p5p5['type'] = '5p5p'
    
    p5p3 = df.groupby(['5p3p', 'sample']).agg( { 'count':'sum' } ).reset_index()
    p5p3 = p5p3.rename(columns = {'5p3p' : 'distance'})
    p5p3['type'] = '5p3p'
    
    res = pd.concat([p5p5,p5p3])
    
    return res


In [26]:
dists = measure_5p3p_distance(filt)

In [ ]:
dists.head()

In [28]:
dists.to_csv("antisense_to_sense_distances.tsv", sep = "\t", header = True, index = False)

In [29]:
# find 5p3p == 8 in parn-1 SG for specific example
df = filt

In [41]:
df['dist5p3p'] = df.apply(lambda row: row['start'] - row['ref_start'] if row['strand'] == "+" else row['ref_end'] - row['end'], axis = 1 )
df.head()
df.query('sample.str.contains("parn1_prg1sg") and dist5p3p == 8').sort_values(by = ['count'], ascending = False).to_csv("prg1SGparn1.tsv", sep = "\t", header = True, index = False)


In [68]:
df[['count', 'sample', 'dist5p3p', 'gene']].groupby(
    ['sample', 'gene', 'dist5p3p']
).agg(
    {'count' : 'sum'}
).reset_index().pivot(
    index = ['dist5p3p', 'gene'], columns = 'sample', values = 'count'
).reset_index().fillna(
    0
).to_csv(
    "distance_5p3p.tsv", sep = "\t", header = True, index = False
)

# 2. Export counts of antisense for each piRNA

In [30]:
counts = filt.groupby(['gene', 'feature', 'sample']).agg({ 'count':'sum' }).reset_index()

In [ ]:
counts.head()

In [ ]:
counts.groupby(['sample','feature']).agg({ 'count':'sum', 'gene':'size'})

In [35]:
counts.to_csv(f"./{outname}.counts.tsv", sep = '\t', header = True, index = False)

# 3. Iterate through filtered bed files and make bigwigs for visualization

In [10]:
# iterate through bedfiles from pipeline output, isolate piRNA mapping reads, make bigwigs
def make_bw(direc, bwdir, chrom_sizes, feat) : 
    
    if not os.path.exists(bwdir) : 
        os.mkdir(bwdir)
        
    files = glob.glob(f"{direc}/*.bed.tsv")
    
    for f in files :
        name = os.path.basename(f).split(".")[0]
        d = pd.read_csv(f, sep = "\t")
        d_sub = d.query('feature == @feat').reset_index(drop = True)
        reformat = d_sub[['chrom', 'start', 'end', 'seq', 'count_total_norm']]
        reformat.to_csv("tmp.bed", sep = "\t", header = False, index = False)

        os.system(f"bash ./bed2bw.sh tmp.bed {bwdir}/{name}.{feat}.bw {chrom_sizes} {feat}")
        
        os.system("rm tmp.bed")

In [5]:
chrom_sizes='/fs/ess/PCON0160/ben/pipelines/nextflow_smRNA/index/bowtie/c_elegans.PRJNA13758.WS279.genomic/c_elegans.PRJNA13758.WS279.genomic_chrom_sizes'

In [ ]:
make_bw("./results_07102023/bed", "./bw22G", chrom_sizes, "siRNA")

In [128]:
chrom_sizes='/fs/ess/PCON0160/ben/pipelines/nextflow_smRNA/index/bowtie/c_elegans.PRJNA13758.WS279.genomic/c_elegans.PRJNA13758.WS279.genomic_chrom_sizes'
bwdir=f'/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_type2_piRNA_08_20_2023/antipiRNA_type2'

def makebws(df, bwdir, chrom_sizes) : 
    
    if not os.path.exists(bwdir) : 
        os.mkdir(bwdir)


    samples = df['sample'].drop_duplicates().tolist() 

    for i,g in enumerate(samples) : 

        sub = df.query('sample == @g')
        feat = sub['feature'].drop_duplicates().tolist()[0]
        name = os.path.basename(g).split(".")[0]
        reformat = sub[['chrom', 'start', 'end', 'seq', 'count']]
        reformat.to_csv("tmp.bed", sep = "\t", header = False, index = False)

        os.system(f"bash ./bed2bw.sh tmp.bed {bwdir}/{name}.{feat}.bw {chrom_sizes} {feat}")
        
        os.system("rm tmp.bed")

        

In [130]:
df = pd.read_csv(f"./antipiRNA_bed_master_filtered_type2.tsv", sep = "\t")
makebws(df, bwdir, chrom_sizes)


The following have been reloaded with a version change:
  1) python/3.9-2022.05 => python/3.6-conda5.2


The following have been reloaded with a version change:
  1) python/3.9-2022.05 => python/3.6-conda5.2


The following have been reloaded with a version change:
  1) python/3.9-2022.05 => python/3.6-conda5.2


The following have been reloaded with a version change:
  1) python/3.9-2022.05 => python/3.6-conda5.2



In [131]:
# iterate through bedfiles from pipeline output, isolate piRNA mapping reads, make bigwigs
def make_sense_bw(direc, bwdir, chrom_sizes) : 
    
    if not os.path.exists(bwdir) : 
        os.mkdir(bwdir)
        
    files = glob.glob(f"{direc}/*.bed.tsv")
    
    for f in files :
        name = os.path.basename(f).split(".")[0]
        d = pd.read_csv(f, sep = "\t")
        d_sub = d.query('feature == "piRNA" and biotype == "redefine_type2_piRNA"').reset_index(drop = True)
        reformat = d_sub[['chrom', 'start', 'end', 'seq', 'count_total_norm']]
        reformat.to_csv("tmp.bed", sep = "\t", header = False, index = False)

        os.system(f"bash ./bed2bw.sh tmp.bed {bwdir}/{name}.piRNA.bw {chrom_sizes} piRNA")
        
        os.system("rm tmp.bed")
    

In [132]:
d="/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_type2_piRNA_08_20_2023/bed"
make_sense_bw(d, f'{d}/piRNA_bw', chrom_sizes)


The following have been reloaded with a version change:
  1) python/3.9-2022.05 => python/3.6-conda5.2


The following have been reloaded with a version change:
  1) python/3.9-2022.05 => python/3.6-conda5.2


The following have been reloaded with a version change:
  1) python/3.9-2022.05 => python/3.6-conda5.2


The following have been reloaded with a version change:
  1) python/3.9-2022.05 => python/3.6-conda5.2



In [6]:
def make_anti_bw(direc, bwdir, chrom_sizes) : 
    
    if not os.path.exists(bwdir) : 
        os.mkdir(bwdir)
        
    files = glob.glob(f"{direc}/*.bed.tsv")
    
    for f in files :
        name = os.path.basename(f).split(".")[0]
        d = pd.read_csv(f, sep = "\t")
        d_sub = d.query('feature == "antipiRNA"').reset_index(drop = True)
        reformat = d_sub[['chrom', 'start', 'end', 'seq', 'count_total_norm']]
        reformat.to_csv("tmp.bed", sep = "\t", header = False, index = False)

        os.system(f"bash ./bed2bw.sh tmp.bed {bwdir}/{name}.antipiRNA.bw {chrom_sizes} antipiRNA")
        
        os.system("rm tmp.bed")

In [ ]:
d="/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_normalize_miRNA_07_31_2023/bed"
make_anti_bw(d, f'{d}/antipiRNA_raw_bw', chrom_sizes)

In [10]:
# merge bigWigs to collapse replicates

def merge_bw(bws, chrom_sizes, name, outdir) :
    
    if not os.path.exists(outdir) : 
        os.mkdir(outdir)
     
    cmd = (f"""sh /fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/merge_bw.sh {chrom_sizes} {name} "{' '.join(bws)}" {outdir} {len(bws)}""")
    os.system(cmd)

In [ ]:
d = "/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_normalize_miRNA_07_31_2023/bed/antipiRNA_raw_bw"
out = './merged_bw_0817_antiraw'
merge_bw(bws = [os.path.join(d, 'parn_YD1.antipiRNA.bw'), 
                os.path.join(d, 'parn_YD2.antipiRNA.bw')], 
        chrom_sizes = chrom_sizes, 
        name = 'parn_YD.antiraw', 
        outdir = out)

merge_bw(bws = [os.path.join(d, 'N2_YD1.antipiRNA.bw'), 
                os.path.join(d, 'N2_YD2.antipiRNA.bw')], 
        chrom_sizes = chrom_sizes, 
        name = 'N2_YD.antiraw', 
        outdir = out)

In [ ]:
d = "/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_07102023/bed/piRNA_bw"
out = './merged_bw_0816'
merge_bw(bws = [os.path.join(d, 'parn_YD1.piRNA.bw'), 
                os.path.join(d, 'parn_YD2.piRNA.bw')], 
        chrom_sizes = chrom_sizes, 
        name = 'parn_YD.piRNA', 
        outdir = out)

merge_bw(bws = [os.path.join(d, 'N2_YD1.piRNA.bw'), 
                os.path.join(d, 'N2_YD2.piRNA.bw')], 
        chrom_sizes = chrom_sizes, 
        name = 'N2_YD.piRNA', 
        outdir = out)

merge_bw(bws = [os.path.join(d, 'parn1_w03del_1_S15_R1_001.piRNA.bw'), 
                os.path.join(d, 'parn1_w03del_2_S16_R1_001.piRNA.bw')], 
        chrom_sizes = chrom_sizes, 
        name = 'parn1w03.piRNA', 
        outdir = out)

d = "/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/antipiRNA_bw_miRNA_norm"
merge_bw(bws = [os.path.join(d, 'parn_YD1.antipiRNA.bw'), 
                os.path.join(d, 'parn_YD2.antipiRNA.bw')], 
        chrom_sizes = chrom_sizes, 
        name = 'parn_YD.antipiRNA', 
        outdir = out)

merge_bw(bws = [os.path.join(d, 'N2_YD1.antipiRNA.bw'), 
                os.path.join(d, 'N2_YD1.antipiRNA.bw')], 
        chrom_sizes = chrom_sizes, 
        name = 'N2_YD.antipiRNA', 
        outdir = out)

merge_bw(bws = [os.path.join(d, 'parn1_w03del_1_S15_R1_001.antipiRNA.bw'), 
                os.path.join(d, 'parn1_w03del_2_S16_R1_001.antipiRNA.bw')], 
        chrom_sizes = chrom_sizes, 
        name = 'parn1w03.antipiRNA', 
        outdir = out)


# 4. calculate length distribution for piRNAs and antisense 

In [141]:
def length_distribution(df) : 
    
    df['first'] = df['seq'].str[0]
    df['length'] = df['seq'].str.len()
    
    df_grouped = df.groupby(['first', 'length', 'feature', 'sample']).agg( {'seq':'count', 'count':'sum'}).reset_index()
    return df_grouped

In [142]:
length_distribution_filt = length_distribution(filt)

In [143]:
length_distribution_raw = length_distribution(piRNA_master_antisense)

In [144]:
length_distribution_raw.head()

,first,length,feature,sample,seq,count
0,A,15,antipiRNA,N2NOT_raw,1,0.242184
1,A,15,antipiRNA,N2PP_raw,5,1.368273
2,A,15,antipiRNA,N2TAP_raw,3,0.425658
3,A,15,antipiRNA,N2_YD1,3,0.233215
4,A,15,antipiRNA,N2_YD2,13,0.375875


In [145]:
length_distribution_raw.to_csv(f"./length_distribution_unfiltered_piRNA.tsv", sep = "\t", header = True, index = False)

# 5. length distribution of sense piRNAs unfiltered


In [4]:
def piRNA_lendist(files) : 

    for e,f in enumerate(files) : 
        print(os.path.basename(f).split(".")[0])
        samples = []
        seqlens = []
        firsts = []
        rpms = []
        features = []
        sample = os.path.basename(f).split(".")[0]
        with open(f, 'r') as f : 
            for line in f : 
                if not line.startswith("chrom") :
                    info = line.strip().split("\t")
                    if info[7] == 'piRNA' or info[7] == "redefine_type2_piRNA" : 
                        seq = str(info[3])
                        seql = len(seq)
                        first = seq[0]
                        rpm = float(info[13])

                        seqlens.append(seql)
                        firsts.append(first)
                        rpms.append(rpm)
                        samples.append(sample)
                        features.append(info[7])
        f.close()

        df = pd.DataFrame({
            'seqlen':seqlens,
            'firstnt':firsts,
            'rpm':rpms,
            'sample':samples,
            'feature' : features
        })  

        if e == 0 : 
            res = df 
        else : 
            res = pd.concat([res, df], ignore_index = True)

    #res_grouped = res.groupby(['firstnt', 'seqlen', 'sample']).agg( {'rpm':'sum'}).reset_index()
    
    return res #res_grouped

In [5]:
files = glob.glob("./results_08_30_2023/bed/*.bed.tsv") #+ glob.glob("./results_07_14_2023/bed/*.bed.tsv") + glob.glob("./results_SNAP_ligation_07_24_2023/bed/*.bed.tsv")

In [6]:
files

['./results_08_30_2023/bed/N2input_raw.trimmed.uniq.x5p4.x3p0.uniq.xartifacts.xc.genome.aligned.v0.m1.combined.bed.tsv',
 './results_08_30_2023/bed/N2prgip_raw.trimmed.uniq.x5p4.x3p0.uniq.xartifacts.xc.genome.aligned.v0.m1.combined.bed.tsv',
 './results_08_30_2023/bed/TW28prgip_raw.trimmed.uniq.x5p4.x3p0.uniq.xartifacts.xc.genome.aligned.v0.m1.combined.bed.tsv',
 './results_08_30_2023/bed/TW29_raw.trimmed.uniq.x5p4.x3p0.uniq.xartifacts.xc.genome.aligned.v0.m1.combined.bed.tsv',
 './results_08_30_2023/bed/TW28TAP_raw.trimmed.uniq.x5p4.x3p0.uniq.xartifacts.xc.genome.aligned.v0.m1.combined.bed.tsv',
 './results_08_30_2023/bed/TW28PP_raw.trimmed.uniq.x5p4.x3p0.uniq.xartifacts.xc.genome.aligned.v0.m1.combined.bed.tsv',
 './results_08_30_2023/bed/TW28input_raw.trimmed.uniq.x5p4.x3p0.uniq.xartifacts.xc.genome.aligned.v0.m1.combined.bed.tsv',
 './results_08_30_2023/bed/N2NOT_raw.trimmed.uniq.x5p4.x3p0.uniq.xartifacts.xc.genome.aligned.v0.m1.combined.bed.tsv',
 './results_08_30_2023/bed/N2TAP_r

In [7]:
lendist = piRNA_lendist(files)

N2input_raw
N2prgip_raw
TW28prgip_raw
TW29_raw
TW28TAP_raw
TW28PP_raw
TW28input_raw
N2NOT_raw
N2TAP_raw
N2PP_raw
TW28NOT_raw


In [8]:
lendist.to_csv("./results_08_30_2023/piRNA_length_distribution.tsv", sep = "\t", header = True, index = False)

In [9]:
lendist

,seqlen,firstnt,rpm,sample,feature
0,20,G,0.879562,N2input_raw,piRNA
1,21,G,17.151459,N2input_raw,piRNA
2,22,G,28.585765,N2input_raw,piRNA
3,23,G,0.219891,N2input_raw,piRNA
4,21,G,0.219891,N2input_raw,piRNA
...,...,...,...,...,...
404692,19,A,1.054166,TW28NOT_raw,piRNA
404693,17,A,0.527083,TW28NOT_raw,piRNA
404694,18,A,2.108331,TW28NOT_raw,piRNA
404695,17,A,1.581249,TW28NOT_raw,piRNA


# 6. make bed for specific examples

In [58]:
# first make master table with all antisense / sense reads for piRNAs
# combine tables
# subset out specific piRNA of interest
# export as bedfile

In [5]:
def make_sense_master(direc) : 
        
    files = glob.glob(f"{direc}/*.bed.tsv")
    
    for e,f in enumerate(files) :
        name = os.path.basename(f).split(".")[0]
        print(name)
        d = pd.read_csv(f, sep = "\t")
        d_sub = d.query('feature == "piRNA"').reset_index(drop = True).drop(columns = ['count'] )
        d_sub = d_sub.rename(columns = {'count_total_norm' : 'count'})
        d_sub['sample'] = name
        if e == 0 : 
            res = d_sub
        else : 
            res = pd.concat([d_sub, res], ignore_index = True)
    return res

In [6]:
sense1 = make_sense_master("./results_07102023/bed")

N2_YD1
N2_YD2
parn1_prg1sg_2_S14_R1_001
parn1_rrf1_redo_1_S21_R1_001
parn1_drh3_2_S20_R1_001
parn_YD2
parn1ego1RNAi_2_S3_R1_001
parn1ego1RNAi_1_S2_R1_001
parn1_prg1DA_2_S12_R1_001
parnOxiYD_2_R1
parn1disl2_2_R1
parn1-ekl1-RNAi-1_S4_R1_001
parn1_prg1DA_1_S11_R1_001
parn1_drh3RNAi_1_S17_R1_001
parn1_prg1sg_1_S13_R1_001
parnOxiYD_1_R1
parn1disl2_1_R1
parn1_drh3RNAi_2_S18_R1_001
parn1_w03del_2_S16_R1_001
parn1_rrf1_2_RENAMED_S19_R1_001
parn1_drh3_RENAMED_1_S22_R1_001
parn1rde3_2_R1
parn1_oma1RNAi_2_S20_R1_001
parn_YD1
parn1-ekl1-RNAi-2_S5_R1_001
parn1_w03del_1_S15_R1_001
parn1rde3_1_R1
parn1_oma1RNAi_1_S19_R1_001


In [7]:
sense2 = make_sense_master("./results_07_14_2023/bed")

TW28NOT_raw
N2NOT_raw
N2input_raw
N2TAP_raw
TW29_raw
N2prgip_raw
TW28TAP_raw
TW28input_raw
TW28PP_raw
TW28prgip_raw
N2PP_raw


In [8]:
anti1 = pd.read_csv("./results_07102023/bed/antipiRNA_bed_master_filtered.tsv", sep = '\t')

In [9]:
anti2 = pd.read_csv("./results_07_14_2023/bed/antipiRNA_bed_master_filtered.tsv", sep = '\t')

In [10]:
anti1 = anti1[['chrom', 'start', 'end', 'seq', 'feature', 'gene', 'sample', 'count']]
anti2 = anti2[['chrom', 'start', 'end', 'seq', 'feature', 'gene', 'sample', 'count']]


In [11]:
sense1 = sense1[['chrom', 'start', 'end', 'seq', 'feature', 'gene', 'sample', 'count']]
sense2 = sense2[['chrom', 'start', 'end', 'seq', 'feature', 'gene', 'sample', 'count']]


In [12]:
master = pd.concat([sense1, anti1, sense2, anti2], ignore_index = True)

In [13]:
master

,chrom,start,end,seq,feature,gene,sample,count
0,I,5330474,5330498,TCAACAATAAAAATTGAGGAGTAG,piRNA,"WBGene00170676,F48C1.10,21ur-12374",parn1_oma1RNAi_1_S19_R1_001,0.140525
1,I,5330474,5330501,TCAACAATAAAAATTGAGGAGTAGATA,piRNA,"WBGene00170676,F48C1.10,21ur-12374",parn1_oma1RNAi_1_S19_R1_001,0.070262
2,I,5330474,5330503,TCAACAATAAAAATTGAGGAGTAGATAGA,piRNA,"WBGene00170676,F48C1.10,21ur-12374",parn1_oma1RNAi_1_S19_R1_001,0.035131
3,I,5431491,5431508,TGCACCTCGTGAGAAGT,piRNA,"WBGene00170835,F27C1.14,21ur-11585",parn1_oma1RNAi_1_S19_R1_001,0.035131
4,I,5486066,5486089,TATTTTGAATAAAATTTTCGGGG,piRNA,"WBGene00174770,C09D4.7,21ur-12380",parn1_oma1RNAi_1_S19_R1_001,0.070262
...,...,...,...,...,...,...,...,...
2507901,IV,16914647,16914666,ATACTTCTATGCATGACAA,antipiRNA,"WBGene00048497,Y116A8C.128,21ur-5307",N2PP_raw,0.273655
2507902,IV,16914649,16914666,ACTTCTATGCATGACAA,antipiRNA,"WBGene00048497,Y116A8C.128,21ur-5307",N2PP_raw,0.273655
2507903,IV,16981618,16981636,ACGTGAATTTGGTGAATA,antipiRNA,"WBGene00050153,Y116A8C.170,21ur-1526",N2PP_raw,0.273655
2507904,IV,17023991,17024010,ATATGGTAAAAAGTTGAAA,antipiRNA,"WBGene00173668,Y116A8C.420,21ur-9954",N2PP_raw,0.273655


In [19]:
#colnames = c("gene", "start", "end", "seq", "rpm", "strand", "sample")

gene = 'WBGene00048837'
samples = ['N2_YD1', 'N2_YD2', 'parn_YD1', 'parn_YD2']
sub = master.query('gene.str.contains(@gene) and sample.isin(@samples)')
sub
sub = sub[['chrom', 'start', 'end', 'seq', 'count', 'feature', 'sample']]

In [20]:
sub

,chrom,start,end,seq,count,feature,sample
381545,IV,15230629,15230658,TCGAATGCATTCTTAGGTACAATGACATT,0.055672,piRNA,parn_YD1
381546,IV,15230630,15230658,TCGAATGCATTCTTAGGTACAATGACAT,0.389701,piRNA,parn_YD1
381547,IV,15230631,15230658,TCGAATGCATTCTTAGGTACAATGACA,0.723731,piRNA,parn_YD1
381548,IV,15230632,15230658,TCGAATGCATTCTTAGGTACAATGAC,1.503134,piRNA,parn_YD1
381549,IV,15230633,15230658,TCGAATGCATTCTTAGGTACAATGA,0.612388,piRNA,parn_YD1
381550,IV,15230634,15230658,TCGAATGCATTCTTAGGTACAATG,0.055672,piRNA,parn_YD1
381551,IV,15230635,15230658,TCGAATGCATTCTTAGGTACAAT,0.111343,piRNA,parn_YD1
381552,IV,15230637,15230658,TCGAATGCATTCTTAGGTACA,0.111343,piRNA,parn_YD1
381553,IV,15230639,15230658,TCGAATGCATTCTTAGGTA,0.055672,piRNA,parn_YD1
381554,IV,15230640,15230658,TCGAATGCATTCTTAGGT,0.222686,piRNA,parn_YD1


In [16]:
specific_example_dir = './specific_examples'

In [17]:
if not os.path.exists(specific_example_dir) : 
    os.mkdir(specific_example_dir)

In [18]:
sub.to_csv(f"./specific_examples/{gene}.bed", sep = '\t', header = True, index = False)